In [32]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [33]:
with open('./datasets/output.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [34]:
num_classes

453

In [35]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [36]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [37]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()



Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_13 (Dense)            (None, 16)                272       
                                                                 
 dense_14 (Dense)            (None, 16)                272       
                                                                 
 dense_15 (Dense)            (None, 453)               7701      
                                                                 
Total params: 24245 (94.71 KB)
Trainable params: 24245 (94.71 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [38]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
15/15 [==============================] - 1s 3ms/step - loss: 6.1190 - accuracy: 0.0000e+00
Epoch 2/500
15/15 [==============================] - 0s 3ms/step - loss: 6.1167 - accuracy: 0.0022
Epoch 3/500
15/15 [==============================] - 0s 3ms/step - loss: 6.1166 - accuracy: 0.0000e+00
Epoch 4/500
15/15 [==============================] - 0s 3ms/step - loss: 6.1166 - accuracy: 0.0022
Epoch 5/500
15/15 [==============================] - 0s 3ms/step - loss: 6.1165 - accuracy: 0.0022
Epoch 6/500
15/15 [==============================] - 0s 4ms/step - loss: 6.1165 - accuracy: 0.0022
Epoch 7/500
15/15 [==============================] - 0s 3ms/step - loss: 6.1164 - accuracy: 0.0022
Epoch 8/500
15/15 [==============================] - 0s 3ms/step - loss: 6.1165 - accuracy: 0.0022
Epoch 9/500
15/15 [==============================] - 0s 2ms/step - loss: 6.1164 - accuracy: 0.0022
Epoch 10/500
15/15 [==============================] - 0s 2ms/step - loss: 6.1163 - accuracy: 0.0022
E

In [39]:
# model.save('Cont-FineTuned.h5')

In [41]:
def preprocess_input(user_input):
    sequence = tokenizer.texts_to_sequences([user_input])
    padded_sequence = pad_sequences(sequence, truncating='post', maxlen=max_len)
    return padded_sequence

# Function to get the predicted answer
def get_predicted_answer(user_input):
    processed_input = preprocess_input(user_input)
    predicted_class = np.argmax(model.predict(processed_input), axis=-1)[0]
    predicted_tag = lbl_encoder.inverse_transform([predicted_class])[0]
    
    for intent in data['intents']:
        if intent['tag'] == predicted_tag:
            responses = intent['responses']
            return np.random.choice(responses)

# Chat loop
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    else:
        response = get_predicted_answer(user_input)
        print("Bot:", response)

1/1 [==============================] - 0s 18ms/step
Bot: The Speaker or the Deputy Speaker not to preside while a resolution for his removal from office is under consideration
(1) At any sitting of the House of the People, while any resolution for the removal of the Speaker from his office is under consideration, the Speaker, or while any resolution for the removal of the Deputy Speaker from his office is under consideration, the Deputy Speaker, shall not, though he is present, preside, and the provisions of clause ( 2 ) of Article 95 shall apply in relation to every such sitting as they apply in relation to a sitting from which the Speaker, or, as the case may be, the Deputy Speaker, is absent
(2) The Speaker shall have the right to speak in, and otherwise to take part in the proceedings of, the House of the People while any resolution for his removal from office is under consideration in the House and shall, notwithstanding anything in Article 100, be entitled to vote only in the fir